<a href="https://colab.research.google.com/github/sarathsoundar/Pyspark_MLlib_Learnings/blob/main/MLlib/Recommendation%20system/recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark


     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 59.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=903bd14b9b36c07d2fbce62ee6c44a639226011f48e48492d2b0710515e2975e
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession



In [3]:
spark = SparkSession.builder.appName('recommender').getOrCreate()

In [4]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [5]:
rating = spark.read.csv('/content/movielens_ratings.csv',inferSchema=True,header=True)

In [6]:
rating.head()

Row(movieId=2, rating=3.0, userId=0)

In [7]:
rating.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [8]:
(training, test) = rating.randomSplit([0.7, 0.2])

In [9]:
als = ALS(maxIter=5, regParam=0.01, rank=4, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

In [10]:
predictions = model.transform(test)


In [11]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      1|   1.0|    26|   2.629873|
|      6|   1.0|     4|  1.2317702|
|      6|   1.0|     9|  1.3570485|
|      6|   1.0|    14|  1.4162165|
|      6|   3.0|    26|   1.827172|
|      3|   1.0|     7|   1.287663|
|      3|   3.0|    14|  1.3901248|
|      5|   1.0|     8|  1.2225988|
|      5|   3.0|    16|  1.6651602|
|      5|   2.0|    26|  2.6781342|
|      4|   1.0|    14|  1.7204988|
|      4|   1.0|    19|  1.7167073|
|      4|   1.0|    24|  1.6420472|
|      2|   1.0|     3| -1.3349912|
|      2|   3.0|     9|  5.3538733|
|      2|   1.0|    12|   3.160547|
|      2|   2.0|    20|   1.936577|
|      2|   1.0|    23|   6.291157|
|      0|   1.0|     3|-0.58655834|
|      0|   1.0|    11| 0.50662184|
+-------+------+------+-----------+
only showing top 20 rows



In [12]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [13]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|     23|    11|
|     25|    11|
|     37|    11|
|     38|    11|
|     43|    11|
|     48|    11|
|     59|    11|
|     62|    11|
|     67|    11|
|     72|    11|
|     81|    11|
|     86|    11|
+-------+------+



In [14]:
reccomendations = model.transform(single_user)
reccomendations.orderBy('prediction',ascending=False).show()


+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     23|    11|   4.618124|
|     38|    11|   4.468296|
|     59|    11|   3.770985|
|     81|    11|  1.6362996|
|     62|    11|  1.5749514|
|     48|    11|  1.5491005|
|     86|    11| 0.99385893|
|      0|    11| 0.50662184|
|     37|    11|-0.65914345|
|     67|    11| -1.8406184|
|     72|    11| -1.9054819|
|     25|    11| -2.7779393|
|     43|    11| -4.0390477|
+-------+------+-----------+

